# For Using PineconeHybridSearchRetriever 
Just run this notebook for trial versions and update the modular code accordingly. For example: instead of using PineconeVectorStore as retriever in RetrieverQA use PineconeHybridSearchRetriever object. Also create new index as embeddings will be stored with sparse matrix encodings.


**Refer this notebook for further details:**

In [ ]:
import os

In [ ]:
%pwd

In [ ]:
os.chdir('../')

In [ ]:
%pwd

In [ ]:
os.environ['PINECONE_API_KEY']=''

In [ ]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
import os

pc = Pinecone(api_key=os.environ.get("PINECONE_API_KEY"))

index_name = 'chatbot'

if index_name not in pc.list_indexes().names():
    pc.create_index(
        name="chatbot",
        dimension=384, 
        metric="dotproduct", 
        spec=ServerlessSpec(
            cloud="aws", 
            region="us-east-1"
        ) 
    ) 

In [ ]:

from langchain_pinecone import PineconeVectorStore
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.embeddings import HuggingFaceEmbeddings
import time

def load_pdf(data):
    loader = DirectoryLoader(data,
                             glob='*.pdf',
                             loader_cls=PyPDFLoader)
    documents = loader.load()
    return documents

extracted_data = load_pdf('data')

def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks = text_splitter.split_documents(extracted_data)
    
    return text_chunks

text_chunks = text_split(extracted_data)

def download_huggingface_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')

    return embeddings

embeddings = download_huggingface_embeddings()

In [ ]:
from pinecone_text.sparse import BM25Encoder

bm25_encoder = BM25Encoder().default()


In [ ]:
bm25_encoder.fit([t.page_content for t in text_chunks])

bm25_encoder.dump('bm25_values.json')

bm25_encoder = BM25Encoder().load('bm25_values.json')

In [ ]:
index = pc.Index("chatbot")

In [ ]:
from langchain.retrievers.pinecone_hybrid_search import PineconeHybridSearchRetriever
retrieval = PineconeHybridSearchRetriever(embeddings=embeddings, 
                                          index=index, 
                                          sparse_encoder=bm25_encoder)

In [ ]:
retrieval

In [ ]:
retrieval.add_texts([t.page_content for t in text_chunks])

In [ ]:
# retrieval.invoke('what is acne')

In [ ]:
from pinecone.grpc import PineconeGRPC as Pinecone
import os

pc = Pinecone(api_key=os.environ.get("PINECONE_API_KEY"))

index = pc.Index("chatbot")
namespace = "default"

In [ ]:
prompt_template="""
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [ ]:
from langchain.prompts import PromptTemplate
PROMPT=PromptTemplate(template=prompt_template, input_variables=["context", "question"])
chain_type_kwargs={"prompt": PROMPT}

In [ ]:
from langchain.chains import RetrievalQA 
from langchain.llms import CTransformers
from langchain_pinecone import PineconeVectorStore
import os

# Initialize a LangChain object for chatting with the LLM
# without knowledge from Pinecone.
llm=CTransformers(model="model/llama-2-7b-chat.ggmlv3.q4_0.bin",
                  model_type="llama",
                  config={'max_new_tokens':512,
                          'temperature':0.8})


# Initialize a LangChain object for chatting with the LLM
# with knowledge from Pinecone. 
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retrieval
)

In [ ]:
user_input='what is acne?'
result=qa({"query": user_input})
print("Response : ", result["result"])